In [1]:
import cv2
from darkflow.net.build import TFNet
import numpy as np
import time
from time import time as timer

import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'

/home/cfchen/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
option = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.2,
    'gpu': 0.85
}

tfnet = TFNet(option)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.05044436454772949s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!

## Without Video Resizing

In [14]:
capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')

def re_scale(capture, percent=30):
    #width = int(frame.shape[1]  * percent / 100)
    #height = int(frame.shape[0] * percent / 100)
    width  =  int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)  * percent / 100)
    height =  int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT) * percent / 100)    
    dim = (width, height)
    return dim

re_scaled = re_scale(capture, 100)
print(re_scaled)

def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('videofile_yolo.avi',fourcc, 30.0, re_scaled)
#out = cv2.VideoWriter('output.avi',fourcc, 30.0, (1920,1080))

print("Video Dim: {}  {}  {}".format(capture.get(3), 'X', capture.get(4)))

colors = [tuple(255 * np.random.rand(3)) for i in range(5)]  #range(100)

(1920, 1080)
Video Dim: 1920.0  X  1080.0


### Masked

In [13]:
start = timer()

while(True):
    ret, frame = capture.read() 

    if ret == True:
        
        '''
        # Re-size
        frame = cv2.resize(frame, (640, 360)) 
        # Re-size
        '''
        
        # B & W image (using red channel only)
        #frame = np.stack((frame[:,:,0],frame[:,:,0],frame[:,:,0]),axis=2)
        results = tfnet.return_predict(frame)
        
        for color, result in zip(colors, results):
            tl = (result['topleft']['x'], result['topleft']['y'])
            br = (result['bottomright']['x'], result['bottomright']['y'])
            brX = result['bottomright']['x']
            brY = result['bottomright']['y']
            label = result['label']
            confidence = result['confidence']
            confidence = round(confidence, 1)
            frame = cv2.rectangle(frame, tl, br, color, 7)
            frame = cv2.putText(frame, label + ' ' + str(confidence), tl, cv2.FONT_HERSHEY_COMPLEX, 0.65, (0, 255, 0), 2)

        out.write(frame)
        cv2.imshow('frame', frame)

        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break
            
    else:
        break
        
print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

print('\nTotal elapsed time = ' + str(timer() - start) + ' s\n')

++++++++ Video End ++++++++

Total elapsed time = 11.933890581130981 s



### With Video Resizing

In [8]:
capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')

re_scaled = re_scale(capture, 30)
print(re_scaled)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('videofile_mask_resized.avi',fourcc, 30.0, re_scaled)
out = cv2.VideoWriter('videofile_yolo_resized.avi',fourcc, 30.0, (640,360))

print("Video Dim: {}  {}  {}".format(capture.get(3), 'X', capture.get(4)))

(576, 324)
Video Dim: 1920.0  X  1080.0


### Masked (Downsample)

In [15]:
start = timer()

i = 0
frame_rate_divider = 3
while(True):
    ret, frame = capture.read() 

    if ret == True:
        if i % frame_rate_divider == 0:
        
            '''
            # Re-size
            frame = cv2.resize(frame, (640, 360)) 
            # Re-size
            '''
        
            results = tfnet.return_predict(frame)
            
            for color, result in zip(colors, results):
                tl = (result['topleft']['x'], result['topleft']['y'])
                br = (result['bottomright']['x'], result['bottomright']['y'])
                brX = result['bottomright']['x']
                brY = result['bottomright']['y']
                label = result['label']
                confidence = result['confidence']
                confidence = round(confidence, 1)
                frame = cv2.rectangle(frame, tl, br, color, 7)
                frame = cv2.putText(frame, label + ' ' + str(confidence), tl, cv2.FONT_HERSHEY_COMPLEX, 0.65, (0, 255, 0), 2)
            
            out.write(frame)
            cv2.imshow('frame', frame)
            i += 1
        else:
            i += 1

        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break
            
    else:
        break
        
print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

print('\nTotal elapsed time = ' + str(timer() - start) + ' s\n')

++++++++ Video End ++++++++

Total elapsed time = 5.291671276092529 s

